# Proses Awal Hingga Akhir

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/PACD/Chest-X-Ray"

import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import json
from skimage.filters import threshold_multiotsu
from skimage.segmentation import watershed
from scipy import ndimage as ndi
from skimage.morphology import h_maxima
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from skimage.feature import hog as skhog
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd

def clahe(img, clipLimit=2, tileGridSize=(8,8)):
  clahe_object = cv2.createCLAHE(clipLimit, tileGridSize)
  return clahe_object.apply(img)

def glcm(img):
   # 1. GLCM Features
    # Hapus [i] dari sini -> gunakan 'masked_img' langsung
    angles = [
        np.pi/4,      # 45°
        np.pi/2,      # 90°
        3*np.pi/4,    # 135°
        5*np.pi/4,    # 225°
        3*np.pi/2,    # 270°
        7*np.pi/4     # 315°
    ]
    glcm_ = graycomatrix(img, distances=[1], angles=angles, levels=256, symmetric=True, normed=True)
    glcm_props = [graycoprops(glcm_, prop)[0, 0] for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']]
    return glcm_props

def hog(img):
  return skhog(img, orientations=9, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=False)

def scale_x(x_train, x_test):
  x_train_np = np.asarray(x_train)
  x_test_np = np.asarray(x_test)
  scaler = MinMaxScaler()
  scaled_train = scaler.fit_transform(x_train_np)
  scaled_test = scaler.transform(x_test_np)
  return scaled_train, scaled_test

def lbp(img):
  lbp_ = local_binary_pattern(img, P=12, R=2, method='uniform')
  lbp_hist, _ = np.histogram(lbp_.ravel(), bins=np.arange(0, 11), range=(0, 10))
  lbp_hist = lbp_hist.astype("float")
  lbp_hist /= (lbp_hist.sum() + 1e-6)
  return lbp_hist

def sobel(img):
  sobel_x = cv2.Sobel(img, cv2.CV_64F, dx=1, dy=0, ksize=3)
  sobel_y = cv2.Sobel(img, cv2.CV_64F, dx=0, dy=1, ksize=3)

  # Magnitude
  sobel_mag = np.sqrt(sobel_x**2 + sobel_y**2)

  # Fitur magnitude
  sobel_mag_mean = np.mean(sobel_mag)
  sobel_mag_std  = np.std(sobel_mag)

  return [sobel_mag_mean, sobel_mag_std]

def split_data(gray_imgs, gray_masks):
  csv = pd.read_csv(path+'/MetaData.csv')
  y = csv['ptb'].values

  x = list(zip(gray_imgs, gray_masks))

  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  lungs_train, masks_train = zip(*x_train)
  lungs_test, masks_test = zip(*x_test)

  return lungs_train, masks_train, y_train, lungs_test, masks_test, y_test

def to_gray_img(img):
  if len(img.shape) == 3 and img.shape[2] == 3:
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  else:
    return img



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# data = np.load(path+'/split_lungs_dataset.npz')
# lung_imgs = np.load(path+"/downsampled_imgs.npy")
# masks_list = np.load(path+"/downsampled_masks.npy")
downsampled_path = "/content/drive/MyDrive/PACD"
lung_imgs = np.load(downsampled_path+"/images_list.npy")
masks_list = np.load(downsampled_path+"/masks_list.npy")

gray_lungs = [to_gray_img(img) for img in lung_imgs]
gray_masks = [to_gray_img(img) for img in masks_list]

gaussian_imgs = [cv2.GaussianBlur(img, (3,3), 0) for img in gray_lungs]
clahe_imgs = [clahe(img) for img in gray_lungs]

lungs_train, masks_train, y_train, lungs_test, masks_test, y_test = split_data(clahe_imgs, gaussian_imgs)

masked_lung_train = [img.copy() for img in lungs_train]
for i in range(len(masked_lung_train)):
  masked_lung_train[i][masks_train[i] == 0] = 0
features_train = [np.hstack([glcm(img), hog(img), lbp(img), sobel(img)]) for img in masked_lung_train]

masked_lung_test = [img.copy() for img in lungs_test]
for i in range(len(masked_lung_test)):
  masked_lung_test[i][masks_test[i] == 0] = 0
features_test = [np.hstack([glcm(img), hog(img), lbp(img), sobel(img)]) for img in masked_lung_test]

scaled_x_train, scaled_x_test = scale_x(features_train, features_test)

In [ ]:
model = SVC(kernel='rbf',class_weight='balanced')
model.fit(scaled_x_train, y_train)

SVC(class_weight='balanced')

In [ ]:
# gaussianBlur > clahe > feature extraction
# Train dan test menggunakan mask bawaan.
# Fitur: glcm(img), hog(img), lbp(img), sobel(img).
# train_test_split(x,y,test_size=0.2,random_state=42)
y_pred = model.predict(scaled_x_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['Normal','TBC']))

[[67  6]
 [22 46]]
              precision    recall  f1-score   support

      Normal       0.75      0.92      0.83        73
         TBC       0.88      0.68      0.77        68

    accuracy                           0.80       141
   macro avg       0.82      0.80      0.80       141
weighted avg       0.82      0.80      0.80       141



# Hasil Tes (Jangan di-run)

In [ ]:
# gaussianBlur > clahe > feature extraction
# Train dan test tidak menggunakan mask
# Fitur: glcm(img), hog(img), lbp(img), sobel(img).
# train_test_split(x,y,test_size=0.2,random_state=42, stratify=y)
y_pred = model.predict(scaled_x_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['Normal','TBC']))

[[62 10]
 [20 49]]
              precision    recall  f1-score   support

      Normal       0.76      0.86      0.81        72
         TBC       0.83      0.71      0.77        69

    accuracy                           0.79       141
   macro avg       0.79      0.79      0.79       141
weighted avg       0.79      0.79      0.79       141



In [ ]:
# clahe > feature extraction
# Train dan test tidak menggunakan mask
# Fitur: glcm(img), hog(img), lbp(img), sobel(img).
# train_test_split(x,y,test_size=0.2,random_state=42, stratify=y)
y_pred = model.predict(scaled_x_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['Normal','TBC']))

[[64  8]
 [20 49]]
              precision    recall  f1-score   support

      Normal       0.76      0.89      0.82        72
         TBC       0.86      0.71      0.78        69

    accuracy                           0.80       141
   macro avg       0.81      0.80      0.80       141
weighted avg       0.81      0.80      0.80       141



In [ ]:
# clahe > feature extraction
# Train dan test menggunakan mask bawaan.
# Fitur: glcm(img), hog(img), lbp(img), sobel(img).
# train_test_split(x,y,test_size=0.2,random_state=42, stratify=y)
y_pred = model.predict(scaled_x_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['Normal','TBC']))

[[56 16]
 [19 50]]
              precision    recall  f1-score   support

      Normal       0.75      0.78      0.76        72
         TBC       0.76      0.72      0.74        69

    accuracy                           0.75       141
   macro avg       0.75      0.75      0.75       141
weighted avg       0.75      0.75      0.75       141



In [ ]:
# clahe > feature extraction
# Train dan test menggunakan mask bawaan.
# fitur: glcm(img), hog(img), lbp(img), sobel(img).
# train_test_split(x,y,test_size=0.2,random_state=42)
y_pred = model.predict(scaled_x_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['Normal','TBC']))

[[59 14]
 [18 50]]
              precision    recall  f1-score   support

      Normal       0.77      0.81      0.79        73
         TBC       0.78      0.74      0.76        68

    accuracy                           0.77       141
   macro avg       0.77      0.77      0.77       141
weighted avg       0.77      0.77      0.77       141



In [ ]:
# clahe > feature extraction
# Train dan test tidak menggunakan mask
# fitur: glcm(img), hog(img), lbp(img), sobel(img).
# train_test_split(x,y,test_size=0.2,random_state=42)
y_pred = model.predict(scaled_x_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['Normal','TBC']))

[[67  6]
 [22 46]]
              precision    recall  f1-score   support

      Normal       0.75      0.92      0.83        73
         TBC       0.88      0.68      0.77        68

    accuracy                           0.80       141
   macro avg       0.82      0.80      0.80       141
weighted avg       0.82      0.80      0.80       141

